In [2]:
import json
import requests
from tqdm import tqdm
import time

In [3]:
def json_load(name):
    with open(f'{name}', 'r', encoding = 'utf-8') as f:
        return json.load(f)
    
def json_save(name, item):
    with open(f'{name}', 'w', encoding = 'utf-8') as f:
        json.dump(item, f, ensure_ascii = False, indent = 4)
        
def read_vanilla(name):
    with open(name) as f:
        data_tmp = f.readlines()

    data = list()
    for q in data_tmp:
        data.append(json.loads(q))
        
    return data

In [30]:
test = read_vanilla("../data/VANILLA/Extended_Dataset_Test.json")

In [32]:
test[0], len(test)

({'answer': 'male',
  'answer_sentence': 'sterjo is a male.',
  'question': 'Which sex does Claude Nicolas Emmery possess ?',
  'question_entity_label': 'Claude Nicolas Emmery',
  'question_id': 58504,
  'question_relation': 'P21'},
 21434,
 107166,
 {'answer': 'Sumi Yukawa',
  'answer_sentence': 'sumi yukawa is the spouse of hideki yukawa',
  'question': 'Who is the spouse of Hideki Yukawa ?',
  'question_entity_label': 'Hideki Yukawa',
  'question_id': 2,
  'question_relation': 'P26'})

In [57]:
qanswer_test_responses = list()
cnt = 0
for q in tqdm(test):
    question = q['question']

    response = requests.get(
        "https://qanswer-core1.univ-st-etienne.fr/api/qa/full?question={0}&lang=en&kb=wikidata".format(question)
    ).json()['queries']
    
    qanswer_test_responses.append({
        'question_id': q['question_id'],
        'response': [{'query': r['query'], 'confidence': r['confidence']} for r in response]
    })
    
    if cnt%50 == 0:
        print(cnt)
        json_save("../processed_data/VANILLA/qanswer_test_responses.json", qanswer_test_responses)
    
    time.sleep(1)
    cnt += 1
    
print(cnt)
json_save("../processed_data/VANILLA/qanswer_test_responses.json", qanswer_test_responses)

  0%|          | 0/5 [00:00<?, ?it/s]

0


100%|██████████| 5/5 [00:11<00:00,  2.22s/it]

5


## Execute each query-candidate and write result

In [23]:
import json
import re
import time
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("https://query.wikidata.org/bigdata/namespace/wdq/sparql")

In [4]:
qanswer_results = json_load("../processed_data/VANILLA/qanswer_test_responses.json")

In [49]:
qanswer_results_new = list()
cnt = 0
for q in qanswer_results[:1]:
    print("---------", q['question_id'], "-----------")

    response = list()

    for r in q['response']:
        try:
            sparql.setQuery(r['query'])
            sparql.setReturnFormat(JSON)
            results = sparql.query().convert()
            result = results["results"]["bindings"]
            time.sleep(0.5)
        except:
            result = list()

        response.append({'query': r['query'], 'confidence': r['confidence'], 'result': result})

    qanswer_results_new.append({'question_id': q['question_id'], 'response': response})
    
    if cnt%50 == 0:
        print("SAVED", cnt)
        json_save("../processed_data/VANILLA/qanswer_test_responses_extended.json", qanswer_results_new)
        
    cnt += 1
    break
    
print("SAVED", cnt)
json_save("../processed_data/VANILLA/qanswer_test_responses_extended.json", qanswer_results_new)

--------- 58504 -----------
0
1
